In [4]:
data_root = 'data/'

import numpy as np
import pandas as pd 
import h5py
import time

train_photo_to_biz = pd.read_csv(data_root+'train_photo_to_biz_ids.csv')
train_labels = pd.read_csv(data_root+'train.csv').dropna()
train_labels['labels'] = train_labels['labels'].apply(lambda x: tuple(sorted(int(t) for t in x.split())))
train_labels.set_index('business_id', inplace=True)
biz_ids = train_labels.index.unique()
print "Number of business: ", len(biz_ids) ,   "(4 business with missing labels are dropped)"

train_folder = data_root+'train_photo_folders/'


## Load image features
f = h5py.File(data_root+'train_image_HOGfeatures.h5','r')
train_image_features = np.copy(f['feature'])
f.close()


t= time.time()
## For each business, compute a feature vector 
df = pd.DataFrame(columns=['business','label','feature vector'])
index = 0
for biz in biz_ids:  
    
    label = train_labels.loc[biz]['labels']
    image_index = train_photo_to_biz[train_photo_to_biz['business_id']==biz].index.tolist()
      
#     print image_index
    image_index = [image for image in image_index if image <len(train_image_features)]
    if len(image_index) == 0:
        continue
    features = train_image_features[image_index]
    mean_feature =list(np.mean(features,axis=0))

    df.loc[index] = [biz, label, mean_feature]
    index+=1
    if index%1000==0:
        print "Buisness processed: ", index, "Time passed: ", "{0:.1f}".format(time.time()-t), "sec"

with open(data_root+"train_biz_HOGfeatures.csv",'w') as f:  
    df.to_csv(f, index=False)

Number of business:  1996 (4 business with missing labels are dropped)
Buisness processed:  1000 Time passed:  4.0 sec


In [5]:
# Check file content
train_business = pd.read_csv(data_root+'train_biz_HOGfeatures.csv')
print train_business.shape
train_business[0:5]

(1115, 3)


,business,label,feature vector
0,1001,"(0, 1, 6, 8)","[0.26290563, 0.094085656, 0.085721985, 0.09975..."
1,100,"(1, 2, 4, 5, 6, 7)","[0.084665455, 0.042148672, 0.098702617, 0.2758..."
2,1006,"(1, 2, 4, 5, 6)","[0.066267937, 0.5848918, 0.31986001, 0.0071875..."
3,101,"(1, 2, 3, 4, 5, 6)","[0.14571482, 0.047799882, 0.06038243, 0.108079..."
4,1012,"(1, 2, 3, 5, 6)","[0.077827349, 0.038681846, 0.044559114, 0.0929..."


## Compute test set businesses

In [6]:
test_photo_to_biz = pd.read_csv(data_root+'test_photo_to_biz.csv')
biz_ids = test_photo_to_biz['business_id'].unique()

## Load image features
f = h5py.File(data_root+'test_image_HOGfeatures.h5','r')
image_filenames = list(np.copy(f['photo_id']))
# print image_filenames
image_filenames = [name.split('/')[-1][:-4] for name in image_filenames]  #remove the full path and the str ".jpg"
image_features = np.copy(f['feature'])
f.close()
# print image_filenames

print "Number of business: ", len(biz_ids)

df = pd.DataFrame(columns=['business','feature vector'])
index = 0
t = time.time()

for biz in biz_ids:     
    
    image_ids = test_photo_to_biz[test_photo_to_biz['business_id']==biz]['photo_id'].tolist()  
#     print image_ids
    image_ids = [id for id in image_ids if str(id) in image_filenames]

    image_index = [image_filenames.index(str(x)) for x in image_ids]
    
#     print image_index
    if len(image_index) == 0:
        continue
    folder = data_root+'test_photo_folders/'            
    features = image_features[image_index]
    mean_feature =list(np.mean(features,axis=0))

    df.loc[index] = [biz, mean_feature]
    index+=1
    if index%1000==0:
        print "Buisness processed: ", index, "Time passed: ", "{0:.1f}".format(time.time()-t), "sec"

with open(data_root+"test_biz_HOGfeatures.csv",'w') as f:  
    df.to_csv(f, index=False)

Number of business:  10000
Buisness processed:  1000 Time passed:  57.3 sec
Buisness processed:  2000 Time passed:  114.6 sec
Buisness processed:  3000 Time passed:  173.1 sec
Buisness processed:  4000 Time passed:  230.3 sec
Buisness processed:  5000 Time passed:  287.6 sec
Buisness processed:  6000 Time passed:  347.1 sec


In [7]:
# Check file content
import pandas as pd
test_business = pd.read_csv(data_root+'test_biz_HOGfeatures.csv')
print test_business.shape
test_business[0:5]

(6196, 2)


,business,feature vector
0,00er5,"[0.14340249, 0.070842437, 0.081278458, 0.18202..."
1,00kad,"[0.14528593, 0.085006081, 0.42187384, 0.176173..."
2,00v0t,"[1.6457032e-15, 0.0, 0.0, 0.0, 1.0971354e-15, ..."
3,01i5j,"[0.27420247, 0.10946717, 0.038950976, 0.057225..."
4,01is9,"[0.054494027, 0.048283879, 0.16143079, 0.41932..."
